In [29]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request

In [30]:
tags = ["gpt-3"] #tags to scrape
years = ['2020'] #years to scrape during
months = ['06', '07'] #months to scrape during (every available day within the month will be scraped)
textName = "mediumText.txt" #name of the output file

#don't touch unless you need to
hdr = {'User-Agent': 'Mozilla/5.0'}


In [31]:
def getArticles(linkz):
    bodyText = ""
    req = Request(linkz,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)  
    title = ''
    title_data = soup.find("article").find_all("h1")
    title = title + title_data[0].text
    description_data = soup.find("article").find_all("p")
    descData = []
    for row in description_data:
        descData.append(row.text)
    descData.pop(0)
    description = ''
    for c in descData:
        description = description + c
    tag_lists = soup.findAll("ul")
    tag_lists.pop(0)
    tagsz = []
    for ul in tag_lists:
        for li in ul.findAll('li'):
            tagsz.append(li.text)
    return [title,description,tagsz]


In [32]:
def scrapeLinksToArticles(tag, years, months):
    startLink = "https://medium.com/tag/"+tag+"/archive/"
    articleLinks = []
    for y in years:
        yearLink = startLink + y
        for m in months:
            monLink = yearLink + "/" + m
            #open the month link and scrape all valid days (days w/ link) into drive
            req = Request(monLink,headers=hdr)
            page = urlopen(req)
            monSoup = BeautifulSoup(page)
            try: #if there are days
                allDays = list(monSoup.find("div", {"class": "col u-inlineBlock u-width265 u-verticalAlignTop u-lineHeight35 u-paddingRight0"}).find_all("div", {"class":"timebucket"}))
                for a in allDays:
                    try: #try to see if that day has a link
                        dayLink = a.find("a")['href']
                        req = Request(dayLink,headers=hdr)
                        page = urlopen(req)
                        daySoup = BeautifulSoup(page)
                        links = list(daySoup.find_all("div", {"class": "postArticle-readMore"}))
                        for l in links:
                            articleLinks.append(l.find("a")['href'])
                    except: pass
            except: #take the month's articles
                links = list(monSoup.find_all("div", {"class": "postArticle-readMore"}))
                for l in links:
                    articleLinks.append(l.find("a")['href'])
                print("issueHere")
    print("Article Links: ", len(articleLinks))
    return articleLinks


In [34]:
articleLinks = []
for tag in tags: 
    articleLinks.extend(scrapeLinksToArticles(tag, years, months))
articleLinks = set(articleLinks) #get rid of any duplicates
articleLinksz = list(articleLinks)
outPutText = open(textName, "a+", encoding='utf8')
count = 0
for art in articleLinksz:
    print(count)
    try:
        outPutText.write(str(getArticles(art)))
    except:
        print("Skipped an error")
    count += 1    
outPutText.close()

issueHere
